# Algoritmo Genético para el Problema del Viajante (TSP)

Implementación de Algoritmos Genéticos (AG) para encontrar una solución subóptima al Problema del Viajante (TSP). El objetivo es encontrar la ruta más corta que visite una lista de ciudades y regrese al punto de partida.

## 🏙️ Clase: Ciudad

Representa una ciudad o punto en el plano cartesiano.

### Métodos:
* `__init__(self, x: float, y: float)`
    * **Descripción:** Constructor de la clase.
    * **Parámetros:** `x` (coordenada X), `y` (coordenada Y).
* `distancia(self, ciudad_destino: 'Ciudad') -> float`
    * **Descripción:** Calcula la distancia euclidiana a otra ciudad.
    * **Parámetros:** `ciudad_destino` (la otra instancia de `Ciudad`).
    * **Retorna:** La distancia euclidiana (float).
* `__repr__(self)`
    * **Descripción:** Representación en string del objeto.
    * **Retorna:** Un string con las coordenadas `(x,y)`.

In [4]:
import random
import numpy as np
import pandas as pd
import operator
import math
from typing import List, Dict, Tuple

class Ciudad:
    # Representa una Ciudad o ciudad en el plano cartesiano.
    # Variables:
    #   - x: Coordenada X del Ciudad.
    #   - y: Coordenada Y del Ciudad.
    
    # Constructor de la clase Ciudad, inicializar con coordenadas x e y.
    def __init__(self, x: float, y: float):
        self.x = x
        self.y = y
     
    # Método para calcular la distancia entre dos ciudades, usando la distancia euclidiana.
    # Hace lo siguiente:
    #   1. Calcula la diferencia absoluta en las coordenadas x e y.
    #   2. Aplica la fórmula de distancia euclidiana.
    # Parámetros: self (Ciudad actual), ciudad_destino (Ciudad al que se calcula la distancia).
    # Retorna: distancia_calculada: Distancia euclidiana entre 2 ciudades (float).   
    def distancia(self, ciudad_destino: 'Ciudad') -> float:

        x_dis = abs(self.x - ciudad_destino.x)
        y_dis = abs(self.y - ciudad_destino.y)
        distancia_calculada = np.sqrt((x_dis ** 2) + (y_dis ** 2))
        
        # distancia_calculada: La distancia euclidiana entre los dos puntos.
        return distancia_calculada

    # Método para representar el objeto Ciudad como una cadena de texto.
    def __repr__(self):
        return f"({self.x},{self.y})"

## Clase: Aptitud (Fitness)

Calcula la aptitud de una ruta (individuo) y su distancia total.

### Métodos:
* `__init__(self, ruta: List[Ciudad])`
    * **Descripción:** Constructor.
    * **Parámetros:** `ruta` (una lista de objetos `Ciudad` que representa un individuo).
* `distanciaRuta(self) -> float`
    * **Descripción:** Calcula la distancia total de la ruta, incluyendo el regreso al inicio. Utiliza memoization (almacena el resultado) para no recalcular.
    * **Retorna:** La distancia total de la ruta (float).
* `rutaApta(self) -> float`
    * **Descripción:** Calcula el valor de aptitud (fitness) de la ruta, que es el inverso de la distancia.
    * **Retorna:** El valor de aptitud (float).

In [5]:
class Aptitud:

    # Constructor de la clase Aptitud, inicializa con una ruta, y distancia y aptitud en 0.
    def __init__(self, ruta: List[Ciudad]):
        self.ruta = ruta
        self.distancia = 0
        self.f_aptitud = 0.0
 
    # Método para calcular la distancia total de la ruta, incluyendo el regreso al punto de inicio.
    # Hace lo siguiente:
    #   1. Si la distancia ya fue calculada (distancia != 0), retorna el valor almacenado.
    #   2. Si no, recorre la ruta sumando las distancias entre ciudades consecutivas.
    #   3. Añade la distancia desde la última ciudad de vuelta a la primera para cerrar el ciclo.
    # Parámetros: self (objeto Aptitud, contiene la ruta).
    # Retorna: distancia_total: La distancia total de la ruta (float).
    def distanciaRuta(self) -> float:

        # Si la distancia es 0, se calcula; de lo contrario, se retorna el valor almacenado.
        if self.distancia == 0:
            distancia_relativa = 0
            num_ciudads = len(self.ruta)
            
            # Recorre todos los pares de ciudades de la ruta
            for i in range(num_ciudads):
                punto_inicial = self.ruta[i]
                
                # punto_final: El siguiente Ciudad en la ruta. Si es el último, regresa al primero (cierra el ciclo).
                punto_final = self.ruta[(i + 1) % num_ciudads]
                
                distancia_relativa += punto_inicial.distancia(punto_final)
            
            # self.distancia: Almacena el resultado para evitar recálculos (memoization).
            self.distancia = distancia_relativa
            
        return self.distancia
 
    # Método para calcular el valor de aptitud (fitness) de la ruta.
    # Hace lo siguiente:
    #   1. Si el valor de aptitud ya fue calculado (f_aptitud != 0), retorna el valor almacenado.
    #   2. Si no, calcula la aptitud como el inverso de la distancia total de la ruta.
    # Parámetros: self (objeto Aptitud, contiene la ruta y distancia).
    # Retorna: f_aptitud: El valor de aptitud (float).
    def rutaApta(self) -> float:
        if self.f_aptitud == 0:
            # self.f_aptitud: Valor de aptitud, se usa 1 / distancia para maximizar la aptitud.
            self.f_aptitud = 1 / float(self.distanciaRuta())
        return self.f_aptitud

## Funciones del Algoritmo Genético

Sección de funciones principales que componen el ciclo de vida del AG.

---

### Función: `crearRuta`

* **Descripción:** Crea una ruta aleatoria (individuo) a partir de la lista de ciudades.
* **Parámetros:** `lista_ciudades` (Lista de objetos `Ciudad` disponibles).
* **Retorna:** Una ruta (permutación aleatoria) como `List[Ciudad]`.

In [6]:
# Método para crear una ruta aleatoria (individuo), dada una lista de ciudades.
# Hace lo siguiente:
#   1. Utiliza random.sample para generar una permutación aleatoria de la lista de ciudades.
def crearRuta(lista_ciudads: List[Ciudad]) -> List[Ciudad]:

    # route: Una permutación aleatoria de la lista de ciudades.
    route = random.sample(lista_ciudads, len(lista_ciudads))
    return route

### Función: `poblacionInicial`

* **Descripción:** Genera la población inicial de rutas (individuos).
* **Parámetros:**
    * `tamano_pob`: Número de individuos (rutas) en la población.
    * `lista_ciudades`: Lista de objetos `Ciudad` para construir las rutas.
* **Retorna:** Una lista de rutas aleatorias que forman la población (`List[List[Ciudad]]`).

In [7]:
# Método para generar la población inicial de rutas.
# Hace lo siguiente:
#   1. Crea una lista vacía para la población.
#   2. Itera 'tamano_pob' veces, creando una ruta aleatoria en cada iteración.
#   3. Añade cada ruta a la población, llamando al método crearRuta.

def poblacionInicial(tamano_pob: int, lista_ciudads: List[Ciudad]) -> List[List[Ciudad]]:
    poblacion = []
    for _ in range(tamano_pob):
        # poblacion: Lista de listas, donde cada lista interna es un individuo (ruta).
        poblacion.append(crearRuta(lista_ciudads))
    return poblacion

### Función: `clasificacionRutas`

* **Descripción:** Evalúa y clasifica todas las rutas en la población según su aptitud (fitness).
* **Parámetros:** `poblacion` (La población actual de rutas).
* **Retorna:** Una lista ordenada de tuplas (`List[Tuple[int, float]]`).
    * Cada tupla contiene: `(índice_de_la_ruta, valor_de_aptitud)`.
    * La lista está ordenada de **mayor a menor** aptitud.

In [8]:

# Método para evaluar y clasificar las rutas en la población según su aptitud.
# Hace lo siguiente:
#   1. Crea un diccionario para almacenar los resultados de aptitud.
#   2. Itera sobre cada ruta en la población, calculando su aptitud usando la clase Aptitud.
#   3. Almacena el índice de la ruta y su aptitud en el diccionario, llamamdo al método rutaApta para obtener el valor.
#   4. Ordena los resultados por aptitud en orden descendente y los retorna como una lista de tuplas.
def clasificacionRutas(poblacion: List[List[Ciudad]]) -> List[Tuple[int, float]]:
    fitness_results = {}
    # fitness_results: Diccionario que mapea el índice de la ruta en la población a su valor de Aptitud.
    for i, ruta in enumerate(poblacion):
        fitness_results[i] = Aptitud(ruta).rutaApta()
        
    # Retorna la lista de resultados ordenada descendentemente por aptitud
    return sorted(fitness_results.items(), key=operator.itemgetter(1), reverse=True)

### Función: `seleccionRutas` (Selección por Ruleta y Elitismo)

* **Descripción:** Selecciona los individuos que pasarán a la siguiente etapa (reproducción). Combina **Elitismo** (conserva a los mejores) y **Selección por Ruleta** (selección proporcional a la aptitud).
* **Parámetros:**
    * `pop_ranked`: La lista de rutas clasificadas (salida de `clasificacionRutas`).
    * `indiv_seleccionados`: El número de individuos que se seleccionan por **elitismo**.
* **Retorna:** Una lista de índices (`List[int]`) de la población que han sido seleccionados como padres.

In [9]:
# --- FUNCIONES DE SELECCIÓN Y APAREAMIENTO ---
# Método para seleccionar los individuos que pasarán a la siguiente etapa de apareamiento.
# Hace lo siguiente: 
#   1. Crea una lista para almacenar los índices de los individuos seleccionados.
#   2. Añade los mejores individuos (elitismo) directamente a la lista de selección.
def seleccionRutas(pop_ranked: List[Tuple[int, float]], indiv_seleccionados: int) -> List[int]:
    resultados_seleccion = []

    for i in range(indiv_seleccionados):
        resultados_seleccion.append(pop_ranked[i][0])

    df = pd.DataFrame(np.array(pop_ranked), columns=["Indice", "Aptitud"])
    df['cum_sum'] = df.Aptitud.cumsum()
    df['cum_perc'] = 100 * df.cum_sum / df.Aptitud.sum()
    

    for _ in range(len(pop_ranked) - indiv_seleccionados):
        seleccion = 100 * random.random() # Número aleatorio entre 0 y 100
        
        # Recorre la ruleta virtual
        for i in range(len(pop_ranked)):
            # Compara el número aleatorio con el porcentaje acumulado (cum_perc)
            if seleccion <= df.iat[i, 3]:
                # Si el número aleatorio cae en el segmento, se selecciona ese individuo.
                resultados_seleccion.append(pop_ranked[i][0])
                break
                
    # resultados_seleccion: Contiene los índices de los padres seleccionados (élite + ruleta).
    return resultados_seleccion

### Función: `grupoApareamiento`

* **Descripción:** Crea el "grupo de apareamiento" (mating pool) extrayendo las rutas completas de la población usando los índices seleccionados.
* **Parámetros:**
    * `poblacion`: La población actual completa.
    * `resultados_seleccion`: Lista de índices (salida de `seleccionRutas`).
* **Retorna:** Una lista de rutas (`List[List[Ciudad]]`) que formarán los pares de apareamiento.

In [10]:
def grupoApareamiento(poblacion: List[List[Ciudad]], resultados_seleccion: List[int]) -> List[List[Ciudad]]:
    grupo_apareamiento = []
    for index in resultados_seleccion:
        grupo_apareamiento.append(poblacion[index])
    return grupo_apareamiento

### Función: `reproduccion` (Cruce OX1)

* **Descripción:** Realiza el cruce entre dos progenitores para generar un hijo. Utiliza el método **Cruce de Orden 1 (Order 1 Crossover - OX1)**.
* **Parámetros:**
    * `progenitor1`: La primera ruta (padre).
    * `progenitor2`: La segunda ruta (madre).
* **Retorna:** La nueva ruta (hijo) generada (`List[Ciudad]`).

In [11]:

def reproduccion(progenitor1: List[Ciudad], progenitor2: List[Ciudad]) -> List[Ciudad]:
    hijo = []
    hijo_p1 = []
    
    tamano_ruta = len(progenitor1)
    
    punto_corte_1 = int(random.random() * tamano_ruta)
    punto_corte_2 = int(random.random() * tamano_ruta)
    
    generacion_inicial = min(punto_corte_1, punto_corte_2)
    generacion_final = max(punto_corte_1, punto_corte_2)

    for i in range(generacion_inicial, generacion_final):
        hijo_p1.append(progenitor1[i])
        
    indices_progenitor2_ciclico = list(range(generacion_final, tamano_ruta)) + list(range(generacion_final))
    
    hijo_p2_ordenado = []
    for index in indices_progenitor2_ciclico:
        ciudad_actual = progenitor2[index]
        if ciudad_actual not in hijo_p1:
            hijo_p2_ordenado.append(ciudad_actual)

    num_antes_corte = generacion_inicial
    ciudads_antes_corte = hijo_p2_ordenado[:num_antes_corte]
    ciudads_despues_corte = hijo_p2_ordenado[num_antes_corte:]
    
    hijo = ciudads_despues_corte + hijo_p1 + ciudads_antes_corte
    
    return hijo

### Función: `reproduccionPoblacion`

* **Descripción:** Genera la nueva población de hijos a partir del grupo de apareamiento.
* **Parámetros:**
    * `grupo_apareamiento`: Lista de rutas que actuarán como padres.
    * `indiv_seleccionados`: Número de individuos que pasan por **elitismo** (se copian sin cruzar).
* **Retorna:** La nueva población (`List[List[Ciudad]]`) compuesta por la élite y los nuevos hijos.

In [12]:

# Método para generar la nueva población de hijos a partir del grupo de apareamiento.
# Hace lo siguiente:
#   1. Copia los mejores individuos directamente (elitismo).
#   2. Genera el resto de la población mediante cruce entre padres.
def reproduccionPoblacion(grupo_apareamiento: List[List[Ciudad]], indiv_seleccionados: int) -> List[List[Ciudad]]:
    hijos = []
    tamano_poblacion = len(grupo_apareamiento)
    
    for i in range(indiv_seleccionados):
        hijos.append(grupo_apareamiento[i])
        
    espacio = random.sample(grupo_apareamiento, tamano_poblacion)
    num_hijos_a_cruzar = tamano_poblacion - indiv_seleccionados

    for i in range(num_hijos_a_cruzar):
        progenitor1 = espacio[i]
        progenitor2 = espacio[tamano_poblacion - i - 1]
        hijo = reproduccion(progenitor1, progenitor2)
        hijos.append(hijo)
        
    return hijos

### Función: `mutacion` (Mutación por Intercambio)

* **Descripción:** Aplica el operador de mutación a un individuo (ruta). Utiliza la **Mutación por Intercambio (Swap Mutation)**.
* **Parámetros:**
    * `individuo`: La ruta (lista de ciudades) a mutar.
    * `razon_mutacion`: La probabilidad de que ocurra un intercambio.
* **Retorna:** El individuo después de la posible mutación (`List[Ciudad]`).

In [13]:
def mutacion(individuo: List[Ciudad], razon_mutacion: float) -> List[Ciudad]:
    for swapped in range(len(individuo)):
        if(random.random() < razon_mutacion):
            swap_with = int(random.random() * len(individuo))
            lugar1 = individuo[swapped]
            lugar2 = individuo[swap_with]
            individuo[swapped] = lugar2
            individuo[swap_with] = lugar1
    return individuo

### Función: `mutacionPoblacion`

* **Descripción:** Aplica el operador de mutación a toda la población de hijos (la nueva generación).
* **Parámetros:**
    * `poblacion`: La población de hijos (generación cruzada).
    * `razon_mutacion`: La probabilidad de mutación.
* **Retorna:** La población mutada (`List[List[Ciudad]]`).

In [14]:
def mutacionPoblacion(poblacion: List[List[Ciudad]], razon_mutacion: float) -> List[List[Ciudad]]:
    pob_mutada = []
    for individuo in poblacion:
        individuo_mutar = mutacion(individuo, razon_mutacion)
        pob_mutada.append(individuo_mutar)
    return pob_mutada

### Función: `nuevaGeneracion`

* **Descripción:** Ejecuta un ciclo completo (una generación) del Algoritmo Genético. Esta función une todos los pasos: Clasificación, Selección, Reproducción y Mutación.
* **Parámetros:**
    * `generacion_actual`: La lista de rutas de la población actual.
    * `indiv_seleccionados`: Número de individuos élite.
    * `razon_mutacion`: Probabilidad de mutación.
* **Retorna:** La nueva población lista para la siguiente iteración (`List[List[Ciudad]]`).

In [15]:
# Método para ejecutar un ciclo completo del Algoritmo Genético.
# Hace lo siguiente:
#   1. Clasifica las rutas por aptitud.
#   2. Selecciona los candidatos para apareamiento.
#   3. Genera el grupo de apareamiento.
#   4. Produce la nueva población mediante cruce.
#   5. Aplica mutaciones a la nueva generación.
# Parámetros:

def nuevaGeneracion(generacion_actual: List[List[Ciudad]], indiv_seleccionados: int, razon_mutacion: float) -> List[List[Ciudad]]:
    pop_ranked = clasificacionRutas(generacion_actual)
    print(f"DEBUG: Mejor aptitud actual: {pop_ranked[0][1]:.6f} (Distancia: {1/pop_ranked[0][1]:.2f})")

    selection_results = seleccionRutas(pop_ranked, indiv_seleccionados)
    grupo_apa = grupoApareamiento(generacion_actual, selection_results)
    hijos = reproduccionPoblacion(grupo_apa, indiv_seleccionados)
    nueva_generacion = mutacionPoblacion(hijos, razon_mutacion)

    return nueva_generacion

##  Función Principal: `algoritmoGenetico`

* **Descripción:** Orquesta todo el proceso del Algoritmo Genético, desde la inicialización hasta el final de las generaciones.
* **Parámetros:**
    * `poblacion`: La lista inicial de objetos `Ciudad`.
    * `tamano_poblacion`: Tamaño constante de la población.
    * `indiv_seleccionados`: Número de individuos de élite.
    * `razon_mutacion`: Probabilidad de mutación.
    * `generaciones`: Número total de generaciones (iteraciones) a ejecutar.
* **Retorna:** La mejor ruta encontrada (`List[Ciudad]`) después de todas las generaciones.

In [16]:
# Método principal que ejecuta el Algoritmo Genético completo.
# Hace lo siguiente:
#   1. Inicializa la población.
#   2. Ejecuta el bucle de generaciones.
#   3. Retorna la mejor ruta encontrada.

def algoritmoGenetico(poblacion: List[Ciudad], tamano_poblacion: int, indiv_seleccionados: int, razon_mutacion: float, generaciones: int) -> List[Ciudad]:
    pop = poblacionInicial(tamano_poblacion, poblacion)
    
    distancia_inicial = 1 / clasificacionRutas(pop)[0][1]
    print(f"--- Inicio del Algoritmo Genético ---")
    print(f"Parámetros: Población={tamano_poblacion}, Élite={indiv_seleccionados}, Mutación={razon_mutacion}")
    print(f"Distancia Inicial (Mejor de la Generación 0): {distancia_inicial:.2f}")
    print("---------------------------------------")
    
    for i in range(generaciones):
        print(f"Generación {i+1}/{generaciones}...")
        pop = nuevaGeneracion(pop, indiv_seleccionados, razon_mutacion)
    
    pop_final_ranked = clasificacionRutas(pop)
    distancia_final = 1 / pop_final_ranked[0][1]
    
    print("---------------------------------------")
    print(f"Distancia Final (Mejor de la Generación {generaciones}): {distancia_final:.2f}")
    
    best_route_index = pop_final_ranked[0][0]
    mejor_ruta = pop[best_route_index]
    print(f"Mejor ruta encontrada: {mejor_ruta}")
    
    return mejor_ruta

## Ejecución del Algoritmo

Aquí definimos la lista de ciudades y ejecutamos el algoritmo genético con los parámetros deseados.

In [26]:
# --- DATOS DE EJEMPLO Y EJECUCIÓN ---
ciudades = [
        Ciudad(x=60.1695, y=24.9354),
        Ciudad(x=41.3784, y=2.1925),
        Ciudad(x=39.4699, y=-0.3774),
        Ciudad(x=43.2630, y=-2.9350),
        Ciudad(x=37.3828, y=-5.9732),
        Ciudad(x=42.8169, y=-1.6493),
    ]

mejor_ruta_final = algoritmoGenetico(
        poblacion=ciudades, 
        tamano_poblacion=100,
        indiv_seleccionados=30,
        razon_mutacion=0.01,
        generaciones=100
    )
print(f"La mejor ruta final es: {mejor_ruta_final}")
    # Demostración del Cruce OX1 corregido
    # print("\n--- Demostración del Cruce OX1 ---")
    # p1 = ciudades
    # p2 = random.sample(ciudades, len(ciudades))
    # print(f"P1: {p1}")
    # print(f"P2: {p2}")
    # h = reproduccion(p1, p2)
    # print(f"Hijo: {h}")
    # print(f"Hijo tiene la longitud correcta: {len(h) == len(p1)}")
    # print(f"Hijo tiene todos los elementos: {len(set(h)) == len(p1)}")

--- Inicio del Algoritmo Genético ---
Parámetros: Población=100, Élite=30, Mutación=0.01
Distancia Inicial (Mejor de la Generación 0): 78.40
---------------------------------------
Generación 1/100...
DEBUG: Mejor aptitud actual: 0.012755 (Distancia: 78.40)
Generación 2/100...
DEBUG: Mejor aptitud actual: 0.012755 (Distancia: 78.40)
Generación 3/100...
DEBUG: Mejor aptitud actual: 0.012755 (Distancia: 78.40)
Generación 4/100...
DEBUG: Mejor aptitud actual: 0.012755 (Distancia: 78.40)
Generación 5/100...
DEBUG: Mejor aptitud actual: 0.012755 (Distancia: 78.40)
Generación 6/100...
DEBUG: Mejor aptitud actual: 0.012755 (Distancia: 78.40)
Generación 7/100...
DEBUG: Mejor aptitud actual: 0.012755 (Distancia: 78.40)
Generación 8/100...
DEBUG: Mejor aptitud actual: 0.012755 (Distancia: 78.40)
Generación 9/100...
DEBUG: Mejor aptitud actual: 0.012755 (Distancia: 78.40)
Generación 10/100...
DEBUG: Mejor aptitud actual: 0.012755 (Distancia: 78.40)
Generación 11/100...
DEBUG: Mejor aptitud actual